<a href="https://colab.research.google.com/github/Olesya002/ML_course/blob/main/10_statement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sklearn
import numpy as np
import scipy

# 1. Загрузим данные об описаниях вакансий и годовых зарплатах

In [ ]:
data = pd.read_csv('salary-train.csv')
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


# 2. Проведем предобработку данных

In [ ]:
# тексты к нижнему регистру
data.FullDescription = data.FullDescription.apply(lambda x: x.lower())
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london ****k ****...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager// luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse / rgn nursing home for young...,Sutton Coldfield,NaN,20355


In [ ]:
# заменим все символы в текстах, кроме букв и цифр на пробелы
import re
data.FullDescription = data.FullDescription.apply(lambda x: re.sub('[^a-zA-Z0-9]', ' ', x))
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,NaN,20355


In [ ]:
# применим TfidfVectorizer (преобразуем тексты в векторы признаков)
from sklearn import feature_extraction

In [ ]:
vectorizer = feature_extraction.text.TfidfVectorizer(min_df = 5)
text_features = vectorizer.fit_transform(data.FullDescription)

In [ ]:
# заменим пропуски в столбцах на строку nan
data['LocationNormalized'].fillna('nan', inplace = True)
data['ContractTime'].fillna('nan', inplace = True)

In [ ]:
# проведем one-hot-кодирование категориальных признаков LocationNormalized и ContractTime
from sklearn.feature_extraction import DictVectorizer

In [ ]:
enc = DictVectorizer()

In [ ]:
one_hot_features = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [ ]:
X_train = scipy.sparse.hstack([text_features, one_hot_features])

In [ ]:
y_train = data.SalaryNormalized

# 3. Обучим ридж-регрессию с параметром alpha = 1

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
a = Ridge(alpha = 1)
a.fit(X_train, y_train)

Ridge(alpha=1)

# 4. Построим прогнозы для двух примеров

In [ ]:
data_test = pd.read_csv('salary-test-mini.csv')
data_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [ ]:
data_test.FullDescription = data_test.FullDescription.apply(lambda x: x.lower())
data_test.FullDescription = data_test.FullDescription.apply(lambda x: re.sub('[^a-zA-Z0-9]', ' ', x))
data_test['LocationNormalized'].fillna('nan', inplace = True)
data_test['ContractTime'].fillna('nan', inplace = True)

In [ ]:
data_test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [ ]:
text_features_1 = vectorizer.transform(data_test.FullDescription)
one_hot_features_1 = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = scipy.sparse.hstack([text_features_1, one_hot_features_1])

In [ ]:
y_test = a.predict(X_test)
y_test

array([56573.53904275, 37196.8221122 ])